## INTRODUCTION

Keras was developed with a focus on enabling fast experimentation. Because of this, it's very user friendly and allows us to go from idea to implementation with only a few steps.<br>

Keras was originally created by François Chollet. Historically, Keras was a high-level API that sat on top of one of three lower level neural network APIs and acted as a wrapper to to these lower level libraries. These libraries were referred to as Keras backend engines.<br>

Now, when you install TensorFlow, you also automatically get Keras, as it is now part of the TensorFlow library.


## Samples And Labels

To train any neural network in a supervised learning task, we first need a data set of samples and the corresponding labels for those samples.

When referring to samples, we're just referring to the underlying data set, where each individual item or data point within that set is called a sample. Labels are the corresponding labels for the samples.

If we were to train a model to do sentiment analysis on headlines from a media source, for example, the corresponding label for each sample headline from the media source could be “positive” or “negative.”

If we were training a model on images of cats and dogs, then the label for each of the images would either be “cat” or “dog.”




## SEQUENTIAL MODEL

## DATA FORMATION AND PREPROCESSING

For this model we will form our own dummy data

In [2]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

### DUMMY STORY BEHIND THE DATA

Suppose that an experimental drug was tested on individuals ranging from age 13 to 100 in a clinical trial. 

The trial had 2100 participants. 

Half of the participants were under 65 years old, and the other half was 65 years of age or older.
 
The trial showed that around 95% of patients 65 or older experienced side effects from the drug. 

Around 95% of patients under 65 experienced no side effects. 

Thus, generally showing that elderly individuals were more likely to experience side effects.

In [3]:
train_labels = []
train_samples = []

#### PERSON WITH SIDE EFFECT === 1 (FOR THE OUTPUT)

#### PERSON WITHOUT SIDE EFFECT === 0 (FOR THE OUTPUT)


TOTAL PEOPLE = 2100<br>
TOTAL YOUNG (>13 and <65) => 2100/2 = 1050<br>
THEREFORE,<br>
  - YOUNG WHO GOT SIDE EFFECTS => 5% of 1050 = 50 (APPROX)<br>
  - YOUNG WHO ARE WITHOUT SIDE EFFECTS => 95% of 1050 = 1000 (APPROX)
        
TOTAL OLD (>=65) => 2100/2 = 1050<br>
THEREFORE,<br>
   - OLD WHO GOT SIDE EFFECTS => 95% of 1050 = 1000 (APPROX)<br>
   - OLD WHO ARE WITHOUT SIDE EFFECTS => 5% of 1050 = 50 (APPROX)       

### DATASET GENERATION CODE

In [4]:
for i in range(50):
    # GENERATE A RANDOM AGE FOR A YOUNG
    random_young_age = randint(13,64)
    train_samples.append(random_young_age)
    # THESE 50 ARE THOSE YOUNG ONES WHO HAVE SIDE EFFECTS
    # THUS, FOR OUTPUT THEY WILL HAVE 1
    train_labels.append(1)
    
    # GENERATE A RANDOM AGE FOR A OLD
    random_old_age = randint(65,100)
    train_samples.append(random_old_age)
    # THESE 50 ARE THOSE OLD ONES WHO DON'T HAVE SIDE EFFECTS
    # THUS, FOR OUTPUT THEY WILL HAVE 0
    train_labels.append(0)
    
    
for i in range(1000):
    # GENERATE A RANDOM AGE FOR A YOUNG
    random_young_age = randint(13,64)
    train_samples.append(random_young_age)
    # THESE 50 ARE THOSE YOUNG ONES WHO DON'T HAVE SIDE EFFECTS
    # THUS, FOR OUTPUT THEY WILL HAVE 0
    train_labels.append(0)
    
    # GENERATE A RANDOM AGE FOR A OLD
    random_old_age = randint(65,100)
    train_samples.append(random_old_age)
    # THESE 50 ARE THOSE OLD ONES WHO HAVE SIDE EFFECTS
    # THUS, FOR OUTPUT THEY WILL HAVE 1
    train_labels.append(1)
    

In [5]:
# VIEW THE AGES WE GENERATED
for age in train_samples:
    print(age)

44
89
43
68
27
85
59
79
35
76
55
99
27
81
38
72
46
67
21
99
41
95
61
80
16
88
22
73
35
69
30
90
40
73
59
96
27
92
34
87
50
69
45
86
61
69
45
98
39
85
17
91
35
73
30
91
19
68
53
87
16
91
31
72
40
95
18
90
38
69
13
100
48
96
23
77
53
99
46
79
49
72
27
83
23
84
35
98
16
100
64
73
18
95
29
89
14
96
33
94
40
72
24
93
17
67
32
69
28
68
27
77
21
69
45
78
60
81
53
89
19
68
55
85
18
85
19
80
38
93
31
85
26
99
47
75
62
94
17
94
38
83
29
83
29
85
51
94
53
84
13
92
27
81
19
99
20
79
38
100
48
81
37
76
30
97
29
75
20
90
22
98
16
99
22
99
24
75
18
65
17
68
39
77
19
70
56
91
62
71
35
72
25
73
27
88
56
97
64
97
60
65
29
77
58
83
52
86
42
78
47
70
40
82
52
72
59
86
21
93
63
69
29
65
39
70
61
76
29
99
31
83
18
95
27
84
59
86
47
74
38
81
19
83
14
84
44
89
60
97
54
77
29
83
49
84
37
91
18
91
32
96
36
94
26
87
45
88
47
87
17
65
42
73
13
77
48
66
33
75
54
88
57
84
18
77
37
65
21
94
39
71
27
90
43
100
22
86
46
78
44
68
52
83
43
87
20
99
36
99
58
83
45
92
39
87
41
83
53
68
46
85
51
96
42
96
23
79
64
93
56
81


In [6]:
# VIEW THE LABELS
for lab in train_labels:
    print(lab)

1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1


WE NEED TO PASS THE DATA TO OUR MODEL IN THE FORM OF A NUMPY ARRAY

THUS, WE WILL NOW CONVERT THE DATASET INTO NUMPY ARRAY FORM

In [8]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

NOW WE WILL SHUFFLE THE DATASET TO GET RID OF ANY ORDER WHICH MAY CAUSE OUR MODEL TO RECOGNIZE ANY IMPROPER PATTERN

In [10]:
train_labels, train_samples = shuffle(train_labels, train_samples)

WE STILL NEED TO DO MORE PRE-PROCESSING ON THE DATA TO DO THINGS LIKE:
- NORMALIZATION
- STANDARDIZATION

THESE THINGS HELP TO TRAIN BETTER AND FASTER

In [12]:
# NOW WE USE THE MINMAXSCALER TO SCALE OUR DATA FROM 0 TO 1 WHICH IS CURRENTLY FROM 13 TO 100

# SCALER OBJECT WILL BE CREATED
scaler = MinMaxScaler(feature_range=(0,1))

# TRANSFORM OUR DATASET TO SCALED DATASET
# THIS FIT_TRANSFORM DOESN'T ACCEPT ANY 1-D DATA
# THUS WE RESHAPE OUR DATA TO BECOME 2-D AND THEN PASS IT TO THE FUNCTION
scale_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

In [14]:
for scale_age in scale_train_samples:
    print(scale_age)

[0.77011494]
[0.37931034]
[0.24137931]
[0.64367816]
[1.]
[0.13793103]
[0.90804598]
[0.97701149]
[0.55172414]
[0.28735632]
[0.59770115]
[0.22988506]
[0.09195402]
[0.70114943]
[0.95402299]
[0.81609195]
[0.08045977]
[0.71264368]
[0.09195402]
[0.83908046]
[0.42528736]
[0.54022989]
[1.]
[0.35632184]
[0.75862069]
[0.45977011]
[0.91954023]
[0.01149425]
[0.1954023]
[0.87356322]
[0.43678161]
[0.87356322]
[0.56321839]
[0.3908046]
[0.57471264]
[0.47126437]
[0.85057471]
[0.67816092]
[0.40229885]
[0.59770115]
[0.72413793]
[0.55172414]
[0.25287356]
[0.01149425]
[0.65517241]
[0.93103448]
[0.24137931]
[0.03448276]
[0.04597701]
[1.]
[0.48275862]
[0.65517241]
[0.01149425]
[0.1954023]
[0.03448276]
[0.70114943]
[0.37931034]
[0.94252874]
[0.01149425]
[0.68965517]
[0.26436782]
[0.79310345]
[0.63218391]
[0.96551724]
[0.24137931]
[0.24137931]
[0.87356322]
[0.06896552]
[0.81609195]
[0.34482759]
[0.48275862]
[0.6091954]
[0.68965517]
[0.04597701]
[0.77011494]
[0.8045977]
[0.09195402]
[0.36781609]
[0.96551724]
[0

[0.05747126]
[0.05747126]
[0.95402299]
[0.62068966]
[0.62068966]
[0.6091954]
[0.72413793]
[0.81609195]
[0.89655172]
[0.06896552]
[0.25287356]
[0.86206897]
[0.88505747]
[0.20689655]
[0.3908046]
[0.01149425]
[0.44827586]
[0.48275862]
[0.10344828]
[0.89655172]
[0.81609195]
[0.2183908]
[0.72413793]
[0.68965517]
[0.16091954]
[0.48275862]
[0.5862069]
[0.52873563]
[0.91954023]
[0.10344828]
[0.20689655]
[0.70114943]
[0.63218391]
[0.35632184]
[0.31034483]
[0.85057471]
[0.59770115]
[0.12643678]
[0.77011494]
[0.57471264]
[0.83908046]
[0.79310345]
[0.89655172]
[0.20689655]
[0.85057471]
[0.98850575]
[0.81609195]
[0.10344828]
[0.20689655]
[0.91954023]
[0.56321839]
[0.87356322]
[0.37931034]
[0.96551724]
[0.73563218]
[0.16091954]
[0.14942529]
[0.68965517]
[0.47126437]
[0.01149425]
[0.82758621]
[0.52873563]
[0.06896552]
[0.37931034]
[0.03448276]
[0.5862069]
[0.94252874]
[0.77011494]
[0.42528736]
[0.28735632]
[0.50574713]
[0.57471264]
[0.2183908]
[0.63218391]
[0.83908046]
[0.40229885]
[0.82758621]
[0.40

[0.75862069]
[0.81609195]
[0.52873563]
[0.89655172]
[0.09195402]
[0.6091954]
[0.8045977]
[0.77011494]
[0.5862069]
[0.87356322]
[0.16091954]
[0.49425287]
[0.24137931]
[0.56321839]
[0.66666667]
[0.85057471]
[0.91954023]
[0.65517241]
[0.66666667]
[0.65517241]
[0.65517241]
[0.83908046]
[0.97701149]
[0.16091954]
[0.89655172]
[0.33333333]
[0.14942529]
[0.79310345]
[0.82758621]
[0.95402299]
[0.73563218]
[0.06896552]
[0.04597701]
[0.70114943]
[1.]
[0.10344828]
[0.18390805]
[0.74712644]
[0.5862069]
[0.2183908]
[0.98850575]
[0.73563218]
[0.71264368]
[0.73563218]
[0.86206897]
[0.48275862]
[0.63218391]
[0.90804598]
[0.50574713]
[0.52873563]
[0.8045977]
[0.75862069]
[0.67816092]
[0.88505747]
[0.43678161]
[0.32183908]
[0.]
[0.14942529]
[0.98850575]
[0.4137931]
[0.62068966]
[0.90804598]
[0.03448276]
[0.72413793]
[0.37931034]
[0.73563218]
[0.91954023]
[0.37931034]
[0.85057471]
[0.13793103]
[0.71264368]
[0.66666667]
[0.66666667]
[0.49425287]
[0.86206897]
[0.89655172]
[0.3908046]
[0.18390805]
[0.2873563

In [20]:
scale_age.shape

(1,)

## THE SEQUENTIAL MODEL COSTRUCTION

SEQUENTIAL MODEL CAN BE DESCRIBED AS A LINEAR STACK OF LAYERS

In [16]:
# IMPORT NECESSARY MODULES
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [19]:
# TO CHECK FOR AVAILABILITY OF A GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("NUMBER OF GPU's : {}".format(len(physical_devices)))

NUMBER OF GPU's : 0


In [21]:
# MODEL CONSTRUCTION

model = Sequential([
    # DENSE LAYER 1
    Dense(units=16, input_shape=(1,), activation='relu'),
    # WE ONLY NEED TO GIVE THE INPUT SHAPE FOR THE FIRST HIDDEN LAYER ONLY
    
    # DENSE LAYER 2
    Dense(units=32, activation='relu'),
    
    # DENSE LAYER 3
    # SOFTMAX GIVES THE PROBABILITY OF THE EACH TYPE OF CATEGORY
    # IN THIS IT TELLS THE PROBABILITY OF 
    # PERSON IS HAVING SIDE EFFECT OR NOT HAVING SIDE EFFECT
    Dense(units=2, activation='softmax')
])

In [22]:
# TO VIEW THE SUMMARY OF MODEL
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


## MODEL TRAINING

In [23]:
# COMPILE OUR MODEL FOR TRAINING
model.compile(
    optimizer = Adam(learning_rate=0.0001),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [24]:
# START TRAINING OUR MODEL
model.fit(
    x = scale_train_samples,
    y = train_labels,
    batch_size = 10,
    epochs = 30,
    # TO SHUFFLE THE DATA
    # BY DEFAULT ALSO THIS PARAMETER IS TRUE ONLY
    shuffle = True,
    # IT ALLOWS US TO SEE THE OUTPUT OF THE PROGRESS
    # IT CAN BE SET TO 0,1,2
    # AT LEVEL 0, THERE IS VERY LESS INFO SHOWN
    # AT 2, THE MAX INFO IS SHOWN
    verbose = 2
)

Epoch 1/30
210/210 - 1s - loss: 0.7202 - accuracy: 0.3814
Epoch 2/30
210/210 - 0s - loss: 0.6796 - accuracy: 0.5676
Epoch 3/30
210/210 - 0s - loss: 0.6362 - accuracy: 0.7129
Epoch 4/30
210/210 - 0s - loss: 0.6028 - accuracy: 0.7529
Epoch 5/30
210/210 - 0s - loss: 0.5742 - accuracy: 0.7805
Epoch 6/30
210/210 - 0s - loss: 0.5470 - accuracy: 0.7995
Epoch 7/30
210/210 - 0s - loss: 0.5196 - accuracy: 0.8171
Epoch 8/30
210/210 - 0s - loss: 0.4920 - accuracy: 0.8286
Epoch 9/30
210/210 - 0s - loss: 0.4668 - accuracy: 0.8467
Epoch 10/30
210/210 - 0s - loss: 0.4435 - accuracy: 0.8529
Epoch 11/30
210/210 - 0s - loss: 0.4224 - accuracy: 0.8714
Epoch 12/30
210/210 - 0s - loss: 0.4032 - accuracy: 0.8724
Epoch 13/30
210/210 - 0s - loss: 0.3864 - accuracy: 0.8829
Epoch 14/30
210/210 - 0s - loss: 0.3718 - accuracy: 0.8929
Epoch 15/30
210/210 - 0s - loss: 0.3591 - accuracy: 0.8967
Epoch 16/30
210/210 - 0s - loss: 0.3482 - accuracy: 0.9019
Epoch 17/30
210/210 - 0s - loss: 0.3385 - accuracy: 0.9067
Epoch 

## USE OF VALIDATION SET

VALIDATION SET CAN BE FORMED USING TWO WAYS :
 - BUILDING IT COMPLETELY FROM SCRATCH JUST LIKE THE TRAINING SET AND PASS TO THE VALIDATAION_DATA PARAMETER IN THE FIT FUNCTION
 - USE THE VALIDATION_SPLIT ATTRIBUTE OF THE FIT FUNCTION
 
IN THIS MODEL WE WILL USE THE SECOND METHOD

In [26]:
model.fit(
    x = scale_train_samples,
    y = train_labels,
    
    # IN THIS VALIDATION SPLIT 
    # WE GAVE 10% OF THE TRAINING SET TO THE VALIDATION SET
    # VALIDATION SET IS NOT USED IN FOR THE TRAINING
    # IT IS ONLY USED FOR VALIDATION
    # IT WILL TAKE THE LAST 10% OF THE TRAINING DATA
    # THE SHUFFLE ONLY HAPPENS AFTER THE SPLIT
    # SO, WE MIGHT SOMETIMES NEED TO SHUFFLE THE DATA EXPLICITLY
    validation_split = 0.1,
    
    batch_size = 10,
    epochs = 30,
    shuffle = True,
    verbose = 2
)

Epoch 1/30
189/189 - 0s - loss: 0.2761 - accuracy: 0.9328 - val_loss: 0.3185 - val_accuracy: 0.9190
Epoch 2/30
189/189 - 0s - loss: 0.2744 - accuracy: 0.9323 - val_loss: 0.3194 - val_accuracy: 0.9000
Epoch 3/30
189/189 - 0s - loss: 0.2731 - accuracy: 0.9312 - val_loss: 0.3172 - val_accuracy: 0.9190
Epoch 4/30
189/189 - 0s - loss: 0.2716 - accuracy: 0.9333 - val_loss: 0.3170 - val_accuracy: 0.9190
Epoch 5/30
189/189 - 0s - loss: 0.2703 - accuracy: 0.9333 - val_loss: 0.3158 - val_accuracy: 0.9190
Epoch 6/30
189/189 - 0s - loss: 0.2692 - accuracy: 0.9333 - val_loss: 0.3147 - val_accuracy: 0.9190
Epoch 7/30
189/189 - 0s - loss: 0.2679 - accuracy: 0.9333 - val_loss: 0.3137 - val_accuracy: 0.9190
Epoch 8/30
189/189 - 0s - loss: 0.2669 - accuracy: 0.9365 - val_loss: 0.3139 - val_accuracy: 0.9190
Epoch 9/30
189/189 - 0s - loss: 0.2660 - accuracy: 0.9333 - val_loss: 0.3130 - val_accuracy: 0.9190
Epoch 10/30
189/189 - 0s - loss: 0.2651 - accuracy: 0.9365 - val_loss: 0.3130 - val_accuracy: 0.9190